In [1]:
import sys
sys.path.append('/lmh_data/work/SEE/train/')

import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
from torch.autograd import Variable
from captum.attr import IntegratedGradients

from dataset import Dataset
from net import define_network

In [2]:
torch.manual_seed(123)
np.random.seed(123)

In [3]:
def evaluate(eval_file, model_file, gene_name):
    eval_set = Dataset(eval_file, gene_name, is_train=True)
    data_loader = data.DataLoader(eval_set, batch_size=1, shuffle=False)

    input_size, output_size = tuple(eval_set[0][0].shape), eval_set[0][1].shape[0]
    patch_size = tuple([int(i / 8) for i in input_size])
    model = torch.nn.DataParallel(define_network(input_size, patch_size, output_size))
    model.load_state_dict(torch.load(model_file))
    model.cuda()
    model.eval()
    ig = IntegratedGradients(model)

    _datas = np.load(eval_file, allow_pickle=True)
    important_genes = dict(Astro=dict(), Endo=dict(), OPC=dict(), ODC=dict())
    for index, batch in enumerate(data_loader, 1):
        if _datas[index-1]['cell_type'] != 'ODC':
            continue
        _important_genes = important_genes[_datas[index-1]['cell_type']]
        input = Variable(batch[0]).cuda().unsqueeze(1)
        target = Variable(batch[1]).cuda().unsqueeze(1) / 10
        
        attributions = torch.zeros(input.reshape(-1).shape).cuda()
        for i in range(target.shape[-1]):
            attributions += ig.attribute(input, target=(0, i))[0, 0].reshape(-1)
        attributions = attributions.cpu().detach().numpy()
        for i in range(len(attributions)):
            if i not in _important_genes.keys():
                _important_genes[i] = 0
            _important_genes[i] += attributions[i]
        
    return important_genes

important_genes = evaluate(
    '/lmh_data/data/sclab/sclab/terminator/eval_dataset.npy',
    '/lmh_data/data/sclab/sclab/terminator/tmp/PDGFRA/model_epoch_9.pth', 'PDGFRA')
important_genes

{'Astro': {},
 'Endo': {},
 'OPC': {},
 'ODC': {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 0.0,
  8: 0.0,
  9: 0.0,
  10: 0.0,
  11: 0.0,
  12: 0.0,
  13: 0.0,
  14: 0.0,
  15: -0.008735756506212056,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: -0.04187459615059197,
  20: -0.012163587380200624,
  21: 0.0,
  22: -0.005311205633915961,
  23: 0.0,
  24: -0.11816312721930444,
  25: 0.0,
  26: -0.03599401121027768,
  27: 0.0,
  28: 0.0,
  29: 0.0,
  30: 0.0,
  31: 0.0,
  32: 0.0,
  33: 0.0,
  34: -0.041534761316142976,
  35: 0.01935940468683839,
  36: 0.0,
  37: -0.04649966803845018,
  38: -0.02989551704376936,
  39: -0.034955314826220274,
  40: 0.03445905586704612,
  41: 0.09784921957179904,
  42: 0.0,
  43: -0.031660970533266664,
  44: 0.0,
  45: -0.1904225079342723,
  46: 0.16918871668167412,
  47: -0.11781119578517973,
  48: 0.012889509787783027,
  49: -0.00844330689869821,
  50: -0.13666742842178792,
  51: 0.022291403263807297,
  52: 0.027090266114100814,
  53: 

In [4]:
scRNA_head = np.load('/lmh_data/data/sclab/sclab/terminator/eval_dataset.npy', allow_pickle=True)[0]['scRNA_head']

def find_special_genes(important_genes, cell_type):
    genes = pd.DataFrame(
        sorted(important_genes[cell_type].items(), key=lambda x: x[1], reverse=True), columns=['index', 'value']
    ).set_index('index')
    genes = genes.sort_values(by=['value'], ascending=False)
    return genes

genes = find_special_genes(important_genes, 'ODC')
genes['name'] = scRNA_head[list(genes.index)]
genes.to_csv('tmp/related_genes_PDGFRA(ODC)_terminator.csv')
genes

,value,name
index,,
18428,76.089651,PCDH9
15794,69.073674,MALAT1
27699,50.117971,IL1RAPL1
7226,34.126830,SPOCK3
28301,33.930064,PLP1
...,...,...
3782,-15.544348,NCKAP5
3848,-15.734166,FMNL2
2239,-16.363069,PPP1R12B
